In [ ]:
#use pyspark dataframe

In [1]:
#pip install praw

In [2]:
import socket
import csv
import pandas as pd
import json
import praw
from bs4 import BeautifulSoup
from pyspark.sql import Row

In [3]:
my_client_id = 'RnK5sYk-GgkzUPk3wHZHsA'
my_client_secret = 'bKaBlHbsCIVYLd3U-xwcqjDWjEIjcw'
my_user_agent = 'example'



reddit = praw.Reddit(client_id=my_client_id,
                     client_secret=my_client_secret,
                     user_agent=my_user_agent)

brands = ['callofdutymobile']

In [4]:
# Building the SparkSession and name
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()                                             

24/04/04 14:49:48 WARN Utils: Your hostname, jupy-06 resolves to a loopback address: 127.0.1.1; using 10.123.51.206 instead (on interface ens18)
24/04/04 14:49:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/04 14:49:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:


all_posts = []

for brand in brands:
    game_subreddit = reddit.subreddit(brand)
    for post in game_subreddit.hot(limit=100):  # Adjust limit as needed
        post.comments.replace_more(limit=None)  # Retrieve all comments
        for comment in post.comments.list():
            row = Row(title=post.title, post_score=post.score, post_id=post.id,
                      subreddit=game_subreddit.display_name, url=post.url,
                      comment=comment.body, comment_score=comment.score,
                      comment_id=comment.id, comment_created=comment.created,
                     reply_number=post.num_comments)
            all_posts.append(row)

# Create DataFrame from list of Rows
df = spark.createDataFrame(all_posts)
 
df.show()
# Concatenate all the rows into a single DataFrame
#df = pd.concat(all_posts, ignore_index=True)

# Print the DataFrame
#df

+--------------------+----------+-------+----------------+--------------------+--------------------+-------------+----------+---------------+------------+
|               title|post_score|post_id|       subreddit|                 url|             comment|comment_score|comment_id|comment_created|reply_number|
+--------------------+----------+-------+----------------+--------------------+--------------------+-------------+----------+---------------+------------+
|Call of Duty: Mob...|         7|1be6f25|callofdutymobile|https://www.reddi...|Please report any...|            1|   kvi01uq|  1.710803096E9|          75|
|Call of Duty: Mob...|         7|1be6f25|callofdutymobile|https://www.reddi...|Hey devs, the new...|           27|   kurfpzv|  1.710375314E9|          75|
|Call of Duty: Mob...|         7|1be6f25|callofdutymobile|https://www.reddi...| First.\n\n#ALCATRAZ|           42|   kur9acw|  1.710372967E9|          75|
|Call of Duty: Mob...|         7|1be6f25|callofdutymobile|https://www.

In [17]:
df.columns

['title',
 'post_score',
 'post_id',
 'subreddit',
 'url',
 'comment',
 'comment_score',
 'comment_id',
 'comment_created',
 'reply_number']

In [18]:
from pyspark.sql.functions import regexp_replace
df = df.withColumn("comment", regexp_replace("comment", "[\n\r]", ""))

In [7]:
df.count()

1359

In [19]:
redditPath = "hdfs://10.123.51.194/user/g23/spark_hdfs_data_crawlingv3"
df.write.option("header",True).csv(redditPath)


In [20]:
rd_df = spark.read.csv("hdfs://10.123.51.194/user/g23/spark_hdfs_data_crawlingv3", header=True)
rd_df.show(20)

+--------------------+----------+-------+----------------+--------------------+--------------------+-------------+----------+---------------+------------+
|               title|post_score|post_id|       subreddit|                 url|             comment|comment_score|comment_id|comment_created|reply_number|
+--------------------+----------+-------+----------------+--------------------+--------------------+-------------+----------+---------------+------------+
|Call of Duty: Mob...|         7|1be6f25|callofdutymobile|https://www.reddi...|Please report any...|            1|   kvi01uq|  1.710803096E9|          75|
|Call of Duty: Mob...|         7|1be6f25|callofdutymobile|https://www.reddi...|Hey devs, the new...|           27|   kurfpzv|  1.710375314E9|          75|
|Call of Duty: Mob...|         7|1be6f25|callofdutymobile|https://www.reddi...|     First.#ALCATRAZ|           42|   kur9acw|  1.710372967E9|          75|
|Call of Duty: Mob...|         7|1be6f25|callofdutymobile|https://www.

In [21]:
rd_df.count()

1359